我如何提取这种文件中的这类图片，以及样本名信息和CN等数据呢？一共有几百页，人工去做提取的耗时很大。我需要从这类pdf文件里提取我所需要的训练模型的图片和信息，如何自动化的完成

# 提取SV断点图
四个SV断点图的坐标，x坐标以染色体区间的两端为准，y坐标以上下两条界线为准; 每一页内的顺序为，先左上 -> 右上 -> 左下 -> 右下
* 此前我只用了几页原PDF文件来确定SV图像的坐标，但当应用于整个pdf文件之后，SV图像的坐标却发生了变化
* 直接使用原PDF文件产生的每一页的图像（是的，PDF子集中同一页坐标和PDF原件同一页坐标是不一样的），来确定每一个需要切割的SV的坐标后运行符合预期
* 建议：根据原PDF文件转换后的图片，确定SV坐标，更改coordinates以及w, h

In [ ]:
from pdf2image import convert_from_path
import cv2
import os

# 将PDF文件转换为图片
def convert_pdf_to_images(pdf_path, output_dir, dpi=300):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    pages = convert_from_path(pdf_path, dpi=dpi)

    for i, page in enumerate(pages):
        page_filename = f"{output_dir}/page_{i+1}.png"
        page.save(page_filename, "PNG")
        print(f"Saved page as image: {page_filename}")

# 根据手动确定的SV图像坐标，为每一页切割出4个SV断点图
def extract_sv_image_from_page(image_path, output_dir, coordinates):
    img = cv2.imread(image_path)

    for idx, (x, y) in enumerate(coordinates):
        w, h = 1050, 320
        cropped_img = img[y:y+h, x:x+w]
        output_filename = f"{output_dir}/{os.path.splitext(os.path.basename(image_path))[0]}_part_{idx+1}.png"
        cv2.imwrite(output_filename, cropped_img)
        print(f"Saved cropped SV image: {output_filename}")


# 设置文件路径
pdf_path = "/Users/xurui/back_up_unit/天津大学文件/本科毕设相关/Article/high-confidence.pdf"
page_output_dir = "/Volumes/T7-shield/CS-Bachelor-Thesis/CNN_model/pdf_convert"
image_output_dir = "/Volumes/T7-shield/CS-Bachelor-Thesis/CNN_model/test-picture"

# 将PDF文件转换为图片
convert_pdf_to_images(pdf_path, page_output_dir)

coordinates = [
    (71, 150),
    (1290, 150),
    (71, 2005),
    (1290, 2005)
]

if not os.path.exists(image_output_dir):
    os.makedirs(image_output_dir)

for page_file in os.listdir(page_output_dir):
    if page_file.endswith(".png"):
        page_path = os.path.join(page_output_dir, page_file)
        extract_sv_image_from_page(page_path, image_output_dir, coordinates)

# 提取文本信息
使用 extract_text_info_from_PDF.py 实现，提取样本名，Position, Oscillating CN （2 and 3 states） CN segments
* 需要注意的地方：
    * 为了避免OCR图片带来的错误，选择直接从PDF文件中提取文本部分
    * 页面之间四个染色体破裂事件的分布，并不是完全一样的（_都是左上角的事件，文本部分坐标kennel有细微差别_），所以划分区间的时候需要注意，可以稍微大一些
* 实现的算法：
    1. 先将每一页的文本都提取出来，包括文本和坐标（放在字典里），坐标用于划分到不同的事件中
    2. 用其他脚本，手动确定每个事件的大概范围，制作文字部分区间
    3. 同一文字区间的文本视为同一个事件的信息
    4. 匹配所需要的信息（例如样本名，POS），方法比较复杂，不同信息提取方式也略有不同
        * 样本名是使用PCAWG提供表格中 donor_idx 进行匹配  
